In [1]:
import os
import glob
import pandas as pd
import lpips
import math
import plotly.express as px
import numpy as np
from natsort import natsorted
import cv2
from PIL import Image
import torchvision.transforms as transforms
from tqdm import tqdm
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import scipy.stats
import matplotlib.pyplot as plt
import scipy.sparse as sp
from scipy.sparse.linalg import spsolve
from scipy.sparse.linalg import cg
from scipy.sparse.linalg import lsqr
from itertools import product
from tqdm import tqdm
import glob
from openpyxl import Workbook
from openpyxl.drawing.image import Image as Image_openpyxl
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment
from openpyxl.drawing.xdr import XDRPositiveSize2D
from openpyxl.utils.units import pixels_to_EMU, cm_to_EMU
from openpyxl.drawing.spreadsheet_drawing import OneCellAnchor, AnchorMarker
import pickle
import plotly.graph_objects as go
import random
from fractions import Fraction

In [2]:
loss_fn_alex = lpips.LPIPS(net='alex')
loss_fn_vgg = lpips.LPIPS(net='vgg')

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/Users/ibk5106/anaconda3/envs/conda_pytorch/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/ibk5106/anaconda3/envs/conda_pytorch/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /Users/ibk5106/anaconda3/envs/conda_pytorch/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


/Users/ibk5106/anaconda3/envs/conda_pytorch/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /Users/ibk5106/anaconda3/envs/conda_pytorch/lib/python3.8/site-packages/lpips/weights/v0.1/vgg.pth


In [3]:
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

transform_f = transforms.ToTensor()

def normalize_image(in_img):
    pixels = np.asarray(in_img).astype('float32')
    pixels = (pixels - mean) / std
    return pixels

In [4]:
limit_frame_count = -1
ROOT = os.path.abspath(os.path.join(os.curdir, '../'))

In [5]:
images_dir = os.path.join(ROOT, 'Dashboard Data/Images')
data_path = os.path.join(ROOT, 'Dashboard Data')
gt_fol = os.path.join(data_path, "GT_N")
avg_typ = 'micro'

In [33]:
object_list = [
    "Accent Paving", "Barrier Post", "Barrier Stump", "Bench", "Bicycle", "Bridge", 
    "Building", "Bus", "Bus Stop", "Car", "Chair", "Closed Sidewalk", "Counter", 
    "Crosswalk", "Curb", "Dog", "Driveway(flat)", "Elevator", "Escalator", "Fence", 
    "Fire hydrant", "Flush Door", "Foldout Sign", "Fountain", "Gate", "Guide dog", 
    "Gutter", "Hose", "Lamp Post", "Mail box", "Maintenance Vehicle", "Motorcycle", 
    "Parallel Parking Spot", "Paratransit vehicle", "Pedestrian Crossing", "Person", 
    "Person with a disability", "Pillar", "Pole", "Puddle", "Push button", "Railing", 
    "Raised Entryway", "Retaining Wall", "Road", "Road Divider", "Road Shoulder", 
    "Roadside Parking", "Sidewalk", "Sidewalk pits", "Sign", "Sign Post", "Sloped Driveway", 
    "Slopped Curb", "Snow", "Stairs", "Stop sign", "Street Vendor", "Table", "Tactile Paving", 
    "Traffic Signals", "Train Platform", "Train Tracks", "Trash bins", "Trash on roads", "Tree", 
    "Turnstile", "Uncontrolled Crossing", "Uneven Stairs", "Unpaved Road", "Unpaved Sidewalk", 
    "Vegetation", "Wall", "Water leakage", "Water Pipes", "Wet surface", "Wheelchair", 
    "White Cane", "Yard Waste"
]
object_list = [ooo.lower() for ooo in object_list]

In [38]:
def calculate_model_ap_ar_af1(gt_path, gt_files, pred_path, obj_list=None):
    gt_dict = {}
    pred_dict = {}

    for fl in gt_files:
        if fl in ["video-10-segment-1.csv", "video-9-segment-2.csv"]:
            continue
        gt_fl = os.path.join(gt_path, fl)
        pred_fl = os.path.join(pred_path, fl)

        if not os.path.exists(pred_fl):
            continue

        if obj_list:
            gt_data = pd.read_csv(gt_fl)
            if limit_frame_count > 0:
                if len(list(gt_data.columns)) > limit_frame_count:
                    gt_data = gt_data.iloc[: , :limit_frame_count]

            gt_data = gt_data.transpose()
            gt_data.columns = [x__.lower() for x__ in gt_data.iloc[0]]
            gt_data = gt_data.reindex(columns=obj_list).iloc[1:].transpose().reset_index()

            pred_data = pd.read_csv(pred_fl)
            if limit_frame_count > 0:
                if len(list(pred_data.columns)) > limit_frame_count:
                    pred_data = pred_data.iloc[: , :limit_frame_count]

            pred_data = pred_data.transpose()
            pred_data.columns = [x__.lower() for x__ in pred_data.iloc[0]]
            pred_data = pred_data.reindex(columns=obj_list).iloc[1:].transpose().reset_index()    
            print(pred_data.isnull().values.any())
            # print(gt_data)
            # print(pred_data)
        else:
            gt_data = pd.read_csv(gt_fl)
            if limit_frame_count > 0:
                if len(list(gt_data.columns)) > limit_frame_count:
                    gt_data = gt_data.iloc[: , :limit_frame_count]
            gt_data.replace(-1, 1, inplace=True)
            pred_data = pd.read_csv(pred_fl)
            if limit_frame_count > 0:
                if len(list(pred_data.columns)) > limit_frame_count:
                    pred_data = pred_data.iloc[: , :limit_frame_count]
            pred_data.replace(-1, 1, inplace=True)
        
        x = []
        for index, row in gt_data.iterrows():
            # if list(row)[0].strip().lower() in ["Sidewalk pits"]:
            #     continue
            if list(row)[0].strip().lower() in gt_dict.keys():
                gt_dict[list(row)[0].strip().lower()] += list(row)[1:]
            else:
                gt_dict[list(row)[0].strip().lower()] = list(row)[1:]

            x.append(len(list(row)[1:]))

        y = []
        for index, row in pred_data.iterrows():
            # if list(row)[0].strip().lower() in ["Sidewalk pits"]:
            #     continue
            if list(row)[0].strip().lower() in pred_dict.keys():
                pred_dict[list(row)[0].strip().lower()] += list(row)[1:]
            else:
                pred_dict[list(row)[0].strip().lower()] = list(row)[1:]
            y.append(len(list(row)[1:]))

        if x != y:
            print(len(x), len(y), fl)

    target_array = []
    pred_array = []

    label_names = []

    for key in gt_dict.keys():
        if key in pred_dict.keys():
            if obj_list:
                if key not in obj_list:
                    continue
            if len(gt_dict[key]) == len(pred_dict[key]):
                target_array.append(gt_dict[key])
                pred_array.append(pred_dict[key])
                label_names.append(key)
            else:
                print(len(gt_dict[key]), len(pred_dict[key]), key)
    
    target_array = np.array(target_array).T
    pred_array = np.array(pred_array).T

    # print(target_array.shape)

    # try:
    if pred_path.endswith('GT_N'):
        precs = precision_score(target_array, pred_array, average=avg_typ)
        recs = recall_score(target_array, pred_array, average=avg_typ)
        f1ss = f1_score(target_array, pred_array, average=avg_typ)
    else:
        precs = precision_score(target_array, pred_array, average=avg_typ)
        recs = recall_score(target_array, pred_array, average=avg_typ)
        f1ss = f1_score(target_array, pred_array, average=avg_typ)
    # except Exception as e:
    #     # ap = 0
    #     print(e)
    #     precs = 0.0
    #     recs = 0.0
    #     f1ss = 0.0

    frm_wise_pn_s = (2*target_array) - pred_array

    return precs, recs, f1ss, frm_wise_pn_s

In [7]:
def get_sim_score(img_dir, vid_n, seg_n):
    images = natsorted(glob.glob(
        os.path.join(
            images_dir,
            f'video-{vid_n}-segment-{seg_n}-frame*'
        )
    ))

    d_s = {}
    
    for f in range(1, len(images)):
        f_now_pth = images[f]
        f_prev_pth = images[f-1]

        frm0, frm1 = (os.path.basename(f_prev_pth).split('-')[-1].split('.')[0], 
                      os.path.basename(f_now_pth).split('-')[-1].split('.')[0])

        image_now = cv2.resize(normalize_image(np.array(Image.open(
            f_now_pth
        ).convert('RGB'))/255), (64, 64), interpolation = cv2.INTER_LINEAR).astype(np.float32)
        image_prev = cv2.resize(normalize_image(np.array(Image.open(
            f_prev_pth
        ).convert('RGB'))/255), (64, 64), interpolation = cv2.INTER_LINEAR).astype(np.float32)

        img0 = transform_f(image_now).unsqueeze(0)
        img1 = transform_f(image_prev).unsqueeze(0)

        d = loss_fn_alex(img0, img1).detach().numpy()[0,0,0,0]

        d_s[f'{frm0}-{frm1}'] = d
    return d_s

In [8]:
coco_common_obj = ['person', 'bicycle', 'car', 'motorcycle', 'bus', 'traffic signals', 'fire hydrant', 'stop sign',
                   'bench', 'dog', 'chair', 'vegetation']

pfb_common_obj = ['road', 'sidewalk', 'tree', 'vegetation', 'building', 'fence', 'traffic signals',
                  'fire hydrant', 'chair', 'trash on roads', 'trash bins', 'person', 'car', 'motorcycle',
                  'bus']

ram_obj_map = {                   
    'chair': 'chair',
    'pillar': 'pillar',
    'table': 'table',
    'person': 'person',
    'man': 'person',
    'building': 'building',
    'city street': 'road',
    'curb': 'curb',
    'pavement': 'sidewalk',
    'road': 'road',
    'car': 'car',
    'snow': 'snow',
    'doorway': 'sloped driveway',
    'elevator': 'elevator',
    'rail': 'train tracks',
    'stair': 'stairs',
    'cane': 'white cane',
    'door': 'flush door',
    'fence': 'fence',
    'barrier': 'barrier post',
    'bench': 'bench',
    'sign': 'sign',
    'bin': 'trash bins',
    'pole': 'pole',
    'street vendor': 'street vendor',
    'blind': 'person with a disability',
    'dog': 'dog',
    'escalator': 'escalator',
    'street sign': 'sign post',
    'bus stop': 'bus stop',
    'railway station': 'train platform',
    'tree': 'tree',
    'traffic light': 'traffic signals',
    'tree trunk': 'tree',
    'recycling bin': 'trash bins',
    'train track': 'train tracks',
    'pedestrian': 'person',
    'bus': 'bus',
    'city bus': 'bus',
    'tour bus': 'bus',
    'wall': 'wall',
    'elevator door': 'elevator',
    'bicycle': 'bicycle',
    'crosswalk': 'crosswalk',
    'decker bus': 'bus',
    'motorcycle': 'motorcycle',
    'motorcyclist': 'person',
    'biker': 'person',
    'motorbike': 'motorcycle',
    'warning sign': 'sign',
    'hydrant': 'fire hydrant',
    'school bus': 'bus',
    'vegetation': 'vegetation',
    'fountain': 'fountain'
}

ram_com_obj = list(set(list(ram_obj_map.values())))
all_com = natsorted(list(set(ram_com_obj) & set(coco_common_obj) & set(pfb_common_obj)))
print(len(all_com))

8


In [9]:
gt_dir = gt_fol # "/Users/imrankabir/Desktop/research/vqa_accessibility/Dashboard-For-VQA/Dashboard Data/GT_N"

gts = natsorted(os.listdir(gt_dir))

# skip_list = [
#     "video-10-segment-1.csv", "video-9-segment-2.csv"
# ]
skip_list = []

gts = [x.split('.')[0] for x in gts if x.endswith('.csv') and int(x.split('-')[1])<=16 and x not in skip_list]#[:1]

# gts = ['video-1-segment-4']

len(gts)

21

In [10]:
models = [
    'GT_N', 'BLIP', 'GPV-1', 'RAM', 'yolo_v7', 'HRNet_V2', 'mask_rcnn', 'faster_rcnn', 'Random', 'ALL_1', 'ALL_0'
]

models_f1 = {}

for model in models:
    print(os.path.join(data_path, model))

    p_, r_, f1_, pn_s_ = calculate_model_ap_ar_af1(
        gt_fol, [f'{gt}.csv' for gt in gts], os.path.join(data_path, model), 
        obj_list=all_com
    )
    models_f1[model] = f1_


/Users/ibk5106/Desktop/research/vqa_accessibility/Dashboard-For-VQA/Dashboard Data/GT_N
/Users/ibk5106/Desktop/research/vqa_accessibility/Dashboard-For-VQA/Dashboard Data/BLIP
/Users/ibk5106/Desktop/research/vqa_accessibility/Dashboard-For-VQA/Dashboard Data/GPV-1
/Users/ibk5106/Desktop/research/vqa_accessibility/Dashboard-For-VQA/Dashboard Data/RAM
/Users/ibk5106/Desktop/research/vqa_accessibility/Dashboard-For-VQA/Dashboard Data/yolo_v7
/Users/ibk5106/Desktop/research/vqa_accessibility/Dashboard-For-VQA/Dashboard Data/HRNet_V2
/Users/ibk5106/Desktop/research/vqa_accessibility/Dashboard-For-VQA/Dashboard Data/mask_rcnn
/Users/ibk5106/Desktop/research/vqa_accessibility/Dashboard-For-VQA/Dashboard Data/faster_rcnn
/Users/ibk5106/Desktop/research/vqa_accessibility/Dashboard-For-VQA/Dashboard Data/Random
/Users/ibk5106/Desktop/research/vqa_accessibility/Dashboard-For-VQA/Dashboard Data/ALL_1
/Users/ibk5106/Desktop/research/vqa_accessibility/Dashboard-For-VQA/Dashboard Data/ALL_0


/Users/ibk5106/anaconda3/envs/conda_pytorch/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
models_f1

{'GT_N': 1.0,
 'BLIP': 0.8222433460076046,
 'GPV-1': 0.7855787476280834,
 'RAM': 0.6573519627411842,
 'yolo_v7': 0.8183381088825215,
 'HRNet_V2': 0.7285877426722497,
 'mask_rcnn': 0.7205623901581723,
 'faster_rcnn': 0.7296703296703296,
 'Random': 0.3337837837837838,
 'ALL_1': 0.41357896915752435,
 'ALL_0': 0.0}

In [12]:
# all_sim_dict = {}

# for gt in gts:
#     v_ = int(gt.split('-')[1])
#     s_ = int(gt.split('-')[3])
#     sims = get_sim_score(images_dir, v_, s_)
#     all_sim_dict[gt] = sims

In [13]:
# import pickle

# with open('all_sims.pickle', 'wb') as handle:
#     pickle.dump(all_sim_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [14]:
def laplace_smoothing(mat, alpha):
    N_t_s = mat.sum(axis=1)
    d_ps = N_t_s.shape[0]
    
    for ind, row in enumerate(mat):
        # new_row = (row + alpha) * (N_t_s[ind] / (N_t_s[ind] + (alpha * d_ps)))
        # new_row = new_row / (np.sum(new_row) + 1e-20)
        new_row = (row + alpha) / (N_t_s[ind] + (alpha * d_ps))
        mat[ind] = new_row

    return mat

In [15]:
def get_transition_matrix(pred_file, objs):
    pred_df = pd.read_csv(pred_file)
    if limit_frame_count > 0:
        if len(list(pred_df.columns)) > limit_frame_count:
            pred_df = pred_df.iloc[: , :limit_frame_count]
    pred_df = pred_df.transpose()
    pred_df.columns = pred_df.iloc[0]
    pred_df = pred_df.iloc[1:]
    pred_df.columns = map(str.lower, pred_df.columns)
    if objs is not None:
        pred_df = pred_df.reindex(columns=objs)

    pred_df = pred_df.fillna('0').transpose()
    
    pred = np.array(pred_df).T
            
    unq_st_str = []
    n_o_f = 2**(len(objs))
    
    for x in range(n_o_f):
        str_bit = str(bin(x)).replace('0b', '').zfill(len(objs))
        unq_st_str.append(str_bit)

    transition_matrix_dict = {
        'st': [x for x in unq_st_str]
    }
    for u_s_s in unq_st_str:
        transition_matrix_dict[u_s_s] = [0.0 for _ in unq_st_str]

    transition_matrix = pd.DataFrame(transition_matrix_dict)
    transition_matrix = transition_matrix.set_index('st')

    act_st = []

    for f in range(1, pred.shape[0]):
        s_prev = ''.join([str(ch) for ch in pred[f-1]])
        s_now = ''.join([str(ch) for ch in pred[f]])
        transition_matrix[s_now][s_prev] += 1.0
        if f == 1:
            act_st.append(int(s_prev, 2))
            act_st.append(int(s_now, 2))
        else:
            act_st.append(int(s_now, 2))

    transition_matrix = np.array(transition_matrix)

    transition_matrix = laplace_smoothing(transition_matrix, alpha=1e-2)  # max([np.max(transition_matrix)/5])

    for tr_ind, tm_row in enumerate(transition_matrix):
        if tm_row.sum() == 0.0:
            transition_matrix[tr_ind] = tm_row + 1/tm_row.shape[0]
            
    return transition_matrix, act_st


def get_transition_matrix_from_arr(pred):
    unq_st_str = []
    n_o_f = 2**(pred.shape[1])
    
    for x in range(n_o_f):
        str_bit = str(bin(x)).replace('0b', '').zfill(pred.shape[1])
        unq_st_str.append(str_bit)

    transition_matrix_dict = {
        'st': [x for x in unq_st_str]
    }
    for u_s_s in unq_st_str:
        transition_matrix_dict[u_s_s] = [0.0 for _ in unq_st_str]

    transition_matrix = pd.DataFrame(transition_matrix_dict)
    transition_matrix = transition_matrix.set_index('st')

    for f in range(1, pred.shape[0]):
        s_prev = ''.join([str(ch) for ch in pred[f-1]])
        s_now = ''.join([str(ch) for ch in pred[f]])
        transition_matrix[s_now][s_prev] += 1.0

    transition_matrix = np.array(transition_matrix)

    transition_matrix = laplace_smoothing(transition_matrix, alpha=1e-2)  # max([np.max(transition_matrix)/5])

    for tr_ind, tm_row in enumerate(transition_matrix):
        if tm_row.sum() == 0.0:
            transition_matrix[tr_ind] = tm_row + 1/tm_row.shape[0]
            
    return transition_matrix

In [16]:
def get_st_p_from_t_mat(transition_matrix):
    I = np.identity(transition_matrix.shape[0])
    P_I = transition_matrix - I
    co_eff = P_I.T

    const = np.array([0.0 for _ in range(co_eff.shape[0])])
    co_eff =  np.append(np.ones((1, co_eff.shape[1])), co_eff, axis=0)
    const = np.append(np.array([1]), const)
    

    try:
        p_s_ifs = lsqr(co_eff, const)[0] # np.linalg.solve(co_eff, const)
    except:
        print(det(co_eff), vid_n, seg_n)
        p_s_ifs = lsqr(co_eff, const)[0] # np.linalg.solve(co_eff, const)

    p_s_ifs = list(p_s_ifs)
    # print(p_s_ifs)
    # for s_ind in range((2**(unq_st[0].shape[0]))-len(p_s_ifs)):
    #     p_s_ifs.append(0.0)

    # p_s_ifs = np.array(p_s_ifs)

    return p_s_ifs

In [17]:
def get_entropy(ss_probs):
    if len(ss_probs) <= 1:
        return 0.0

    tot_ss_ent = 0

    for prb in ss_probs:
        if prb == 0:
            log_p_ss = 0
        else:
            try:
                log_p_ss = math.log2(prb)
            except:
                log_p_ss = 0.0 # math.log2(prb+1e-15)

        t_ent = - prb * log_p_ss

        tot_ss_ent = tot_ss_ent + t_ent

    tot_ss_ent = tot_ss_ent / math.log2(len(ss_probs))

    return tot_ss_ent

In [18]:
def get_emission_matrix(gt_file, objs):
    gt_df = pd.read_csv(gt_file)
    if limit_frame_count > 0:
        if len(list(gt_df.columns)) > limit_frame_count:
            gt_df = gt_df.iloc[: , :limit_frame_count]
    gt_df = gt_df.transpose()
    gt_df.columns = gt_df.iloc[0]
    gt_df = gt_df.iloc[1:]
    gt_df.columns = map(str.lower, gt_df.columns)
    if objs is not None:
        gt_df = gt_df.reindex(columns=objs)

    gt_df = gt_df.fillna('0').transpose()

    all_states = []
    n_o_s = 2**(len(objs))

    for x in range(n_o_s):
        str_bit = str(bin(x)).replace('0b', '').zfill(len(objs))
        all_states.append(str_bit)

    emission_matrix_dict = {
        'st': [x for x in all_states]
    }
    for u_f, _ in enumerate(range(len(list(gt_df.columns))+1)):
        emission_matrix_dict[str(u_f)] = [0.0 for _ in all_states]

    emission_matrix = pd.DataFrame(emission_matrix_dict)
    emission_matrix = emission_matrix.set_index('st') 

    frame_state_dict = {}
    for col in gt_df.columns:
        frm_df = gt_df.reindex(columns=[col])
        c_state = "".join([str(x) for x in np.array(frm_df)[:, 0]])
        c_f = col.replace('_', '-').split('-')[-1]
        if c_state not in frame_state_dict.keys():
            frame_state_dict[c_state] = []
        frame_state_dict[c_state].append(c_f)

    for f_st in all_states:
        if f_st not in frame_state_dict.keys():
            oth_prob = 0.1
            prob_each = oth_prob / len(list(gt_df.columns))
            emission_matrix[str(len(list(gt_df.columns)))][f_st] = 1.0 - oth_prob
            for f_f, _ in enumerate(gt_df.columns):
                emission_matrix[str(f_f)][f_st] = prob_each
        else:
            prob_each = 1 / len(frame_state_dict[f_st])
            for f_f in frame_state_dict[f_st]:
                emission_matrix[f_f][f_st] = prob_each

    emission_matrix = np.array(emission_matrix)

    emission_matrix = laplace_smoothing(emission_matrix, alpha=1e-3)

    return emission_matrix

In [19]:
def get_inital_prob_dist_and_obsrv_seq(pred_file, objs):
    pred_df = pd.read_csv(pred_file)
    if limit_frame_count > 0:
        if len(list(pred_df.columns)) > limit_frame_count:
            pred_df = pred_df.iloc[: , :limit_frame_count]
    pred_df = pred_df.transpose()
    pred_df.columns = pred_df.iloc[0]
    pred_df = pred_df.iloc[1:]
    pred_df.columns = map(str.lower, pred_df.columns)
    if objs is not None:
        pred_df = pred_df.reindex(columns=objs)

    pred_df = pred_df.fillna('0').transpose()
    
    pred = np.array(pred_df).T
            
    unq_st_str = []
    n_o_f = 2**(len(objs))
    
    for x in range(n_o_f):
        str_bit = str(bin(x)).replace('0b', '').zfill(len(objs))
        unq_st_str.append(str_bit)

    init_matrix_dict = {
        'st': ['init']
    }
    
    for u_s_s in unq_st_str:
        init_matrix_dict[u_s_s] = [0.0]

    init_matrix = pd.DataFrame(init_matrix_dict)
    init_matrix = init_matrix.set_index('st')

    
    s_init = ''.join([str(ch) for ch in pred[0]])
    init_matrix[s_init]['init'] += 1.0

    init_matrix = np.array(init_matrix)

    init_matrix = laplace_smoothing(init_matrix, alpha=1e-4)  # max([np.max(init_matrix)/5, 1e-5])

    for tr_ind, tm_row in enumerate(init_matrix):
        if tm_row.sum() == 0.0:
            init_matrix[tr_ind] = tm_row + 1/tm_row.shape[0]

    obsrv_seq = np.array([f for f in range(0, pred.shape[0])])        
            
    return init_matrix, obsrv_seq

In [20]:
def viterbi(y, A, B, pi):
    """
        viterbi algorithm
        :param y: observation sequence
        :param A: the transition matrix
        :param B: the emission matrix
        :param pi: the initial probability distribution
    """
    A = -np.log(A)
    B = -np.log(B)
    pi = -np.log(pi)
    
    N = B.shape[0]
    x_seq = np.zeros([N, 0])
    V = B[:, y[0]] + pi

    # forward to compute the optimal value function V
    for y_ in y[1:]:
        _V = np.tile(B[:, y_], reps=[N, 1]).T + A.T + np.tile(V, reps=[N, 1])
        x_ind = np.argmin(_V, axis=1)
        x_seq = np.hstack([x_seq, np.c_[x_ind]])
        V = _V[np.arange(N), x_ind]

    x_T = np.argmin(V)
    max_P = V[x_T]

    # backward to fetch optimal sequence
    x_seq_opt, i = np.zeros(x_seq.shape[1]+1), x_seq.shape[1]
    prev_ind = x_T

    while i >= 0:
        x_seq_opt[i] = prev_ind
        i -= 1
        prev_ind = x_seq[int(prev_ind), i]
    return x_seq_opt, max_P

def viterbi_bk(y, A, B, pi):
    """
        viterbi algorithm
        :param y: observation sequence
        :param A: the transition matrix
        :param B: the emission matrix
        :param pi: the initial probability distribution
    """
    # A = np.log10(A)
    # B = np.log10(B)
    # pi = np.log10(pi)
    
    N = B.shape[0]
    x_seq = np.zeros([N, 0])
    V = B[:, y[0]] * pi

    # forward to compute the optimal value function V
    for y_ in y[1:]:
        _V = np.tile(B[:, y_], reps=[N, 1]).T * A.T * np.tile(V, reps=[N, 1])
        x_ind = np.argmax(_V, axis=1)
        x_seq = np.hstack([x_seq, np.c_[x_ind]])
        V = _V[np.arange(N), x_ind]

    x_T = np.argmax(V)
    max_P = V[x_T]

    # backward to fetch optimal sequence
    x_seq_opt, i = np.zeros(x_seq.shape[1]+1), x_seq.shape[1]
    prev_ind = x_T

    while i >= 0:
        x_seq_opt[i] = prev_ind
        i -= 1
        prev_ind = x_seq[int(prev_ind), i]
    return x_seq_opt, max_P


def viterbi_2(y, A, B, pi, x):
    """
        viterbi algorithm
        :param y: observation sequence
        :param A: the transition matrix
        :param B: the emission matrix
        :param pi: the initial probability distribution
        :param x: model heat map sequence 
    """
    current_prob = pi[0, x[0]] * B[x[0]][0]

    for j, x_ in enumerate(x[1:]):
        # print(A[x[j]][x_], B[x_][j+1], x_, j+1)
        current_prob = current_prob * A[x[j]][x_] * B[x_][j+1]
        
    return x, -np.log(current_prob)

In [21]:
def hamming_dist(s1, s2):
    assert len(s1) == len(s2)
    return sum(c1 != c2 for c1, c2 in zip(s1, s2))

def calc_path_dist(pth1, pth2):
    tot_h_dist = 0
    all_p_dists__ = []
    for p_c, p_ in enumerate(pth1):
        h_dist = hamming_dist(p_, pth2[p_c]) / len(p_)
        all_p_dists__.append(h_dist)
        tot_h_dist = tot_h_dist + h_dist
    dist_pth_ = tot_h_dist / len(pth1)
    return dist_pth_, np.array(all_p_dists__)

In [22]:
def ApEn(U, m, r):
    def _maxdist(x_i, x_j):
        return max([abs(ua - va) for ua, va in zip(x_i, x_j)])
    
    def _phi(m):
        x = [[U[j] for j in range(i, i + m - 1 + 1)] for i in range(N - m + 1)]
        C = [
            len([1 for x_j in x if _maxdist(x_i, x_j) <= r]) / (N - m + 1.0)
            for x_i in x
        ]
        return (N - m + 1.0) ** (-1) * sum(np.log(C))
    
    N = len(U)
    
    return abs(_phi(m + 1) - _phi(m))

In [23]:
def gen_heat_map_from_arr(gt_data, pred_data, obj_list=None, d_s=[]):
    hm_ = (gt_data*2) - pred_data

    color_tp = 'rgb(71, 137, 209)' 
    color_tn = 'rgb(104, 212, 202)'
    color_fp = 'rgb(202, 121, 59)' 
    color_fn = 'rgb(198, 59, 59)'
    
    fig = go.Figure()
    
    colorscale = [
        [0, color_fp],
        [0.33333, color_tn],
        [0.66667, color_tp],
        [1, color_fn]
    ]
    
    fig.update_layout(
        template="simple_white",
        xaxis=dict(title_text="Frames", tickmode='linear'),
        yaxis=dict(title_text="Objects",  tickmode='linear'),
        width=1000,
        height=250,
        margin=dict(l=0, r=0, t=10, b=5),
        title=dict(
            text="",
            font=dict(size=16, color="black"),
            x=0.5,
            y=0.995
    
        )
    )
    
    fig.add_trace(
        go.Heatmap(
            z=hm_,
            x=[f'{i} {d_s[i]:.2f}' for i in range(hm_.shape[1])],
            y=[y.capitalize() for i, y in enumerate(obj_list)],
            colorscale=colorscale,
            zmin=-1,
            zmax=2
        )
    )


    # fig.add_traces(go.Bar(x=[], y =[], name='TP',
    #                  marker_color=color_tp))
    # fig.add_traces(go.Bar(x=[], y =[], name='FP',
    #                  marker_color=color_fp))
    # fig.add_traces(go.Bar(x=[], y =[], name='TN',
    #                  marker_color=color_tn))
    # fig.add_traces(go.Bar(x=[], y =[], name='FN',
    #                  marker_color=color_fn))
    
    fig.update_layout(
        font_family="Courier New, monospace",
        font_color="black",
        title_font_family="Courier New, monospace",
        title_font_color="black",
        legend_title_font_color="black",
        font=dict(size=14),
    )

    fig.update_traces(
      colorbar_tickmode='array',
      colorbar_ticktext=['FP', 'TN', 'TP', 'FN'],
      colorbar_tickvals=[-1, 0, 1, 2],
      colorbar_tickangle=0,
      selector=dict(type='heatmap')
     )

    fig.update_xaxes(ticklabelposition='outside')
    
    fig.update_xaxes(
        tickson="boundaries",
        ticklen=5,
        tickangle=90
    )
    fig.update_layout(
         xaxis=dict(showgrid=True),
         yaxis=dict(showgrid=True)
    )
    
    fig.update_layout(xaxis=dict(domain=[0, 1]))
    
    hm_figs = fig

    return hm_figs

In [24]:
def gen_heat_map_model_only_from_arr(pred_data, obj_list=None, d_s=[]):
    hm_ = pred_data # gt_data.mul(2).add(pred_data.mul(-1), fill_value=0)

    color_tp = 'rgb(71, 137, 209)' 
    color_tn = 'rgb(104, 212, 202)'
    color_fp = 'rgb(202, 121, 59)' 
    color_fn = 'rgb(198, 59, 59)'
    color_agreement = 'rgb(6, 200, 115)' # green
    color_disagreement = 'rgb(211, 6, 50)'
    
    fig = go.Figure()
    
    colorscale = [
        [0, color_disagreement],
        [1, color_agreement]
    ]
    
    fig.update_layout(
        template="simple_white",
        xaxis=dict(title_text="Frames", tickmode='linear'),
        yaxis=dict(title_text="Objects",  tickmode='linear'),
        width=1000,
        height=250,
        margin=dict(l=0, r=0, t=10, b=5),
        title=dict(
            text="",
            font=dict(size=16, color="black"),
            x=0.5,
            y=0.995
    
        )
    )
    
    fig.add_trace(
        go.Heatmap(
            z=hm_,
            x=[f'{i} {d_s[i]:.2f}' for i in range(hm_.shape[1])],
            y=[y.capitalize() for i, y in enumerate(obj_list)],
            colorscale=colorscale,
            zmin=0,
            zmax=1
        )
    )

    # fig.add_traces(go.Bar(x=[], y =[], name='TP',
    #                  marker_color=color_tp))
    # fig.add_traces(go.Bar(x=[], y =[], name='FP',
    #                  marker_color=color_fp))
    # fig.add_traces(go.Bar(x=[], y =[], name='TN',
    #                  marker_color=color_tn))
    # fig.add_traces(go.Bar(x=[], y =[], name='FN',
    #                  marker_color=color_fn))
    
    fig.update_layout(
        font_family="Courier New, monospace",
        font_color="black",
        title_font_family="Courier New, monospace",
        title_font_color="black",
        legend_title_font_color="black",
        font=dict(size=14),
    )

    fig.update_traces(
      colorbar_tickmode='array',
      colorbar_ticktext=['Disagree', 'Agree'],
      colorbar_tickvals=[0, 1],
      colorbar_tickangle=0,
      selector=dict(type='heatmap')
     )

    fig.update_xaxes(ticklabelposition='outside')
    
    fig.update_xaxes(
        tickson="boundaries",
        ticklen=5,
        tickangle=90
    )
    fig.update_layout(
         xaxis=dict(showgrid=True),
         yaxis=dict(showgrid=True)
    )
    
    fig.update_layout(xaxis=dict(domain=[0, 1]))
    
    hm_figs = fig

    return hm_figs

In [25]:
def get_vetical_axis_lines(x_labels, cg):

    vertical_lines = []

    for i in range(len(x_labels)):
        vertical_lines.append({
            'type': 'line',
            'x0': i / len(x_labels),
            'x1': i / len(x_labels),
            'y0': 0,
            'y1': 1,
            'xref': 'paper',  
            'yref': 'paper',  
            'line': {
                'color': cg, 
                'width': 2,  
            },
            'opacity': 0.5
    })
        
    return vertical_lines


def get_horizontal_axis_lines(y_labels, cg):

    horizontal_lines = []

    for i in range(len(y_labels) + 1):
        horizontal_lines.append({
            'type': 'line',
            'x0': 0,
            'x1': 1,
            'y0': i / len(y_labels),
            'y1': i / len(y_labels),
            'xref': 'paper',
            'yref': 'paper', 
            'line': {
                'color': cg, 
                'width': 2, 
            },
            'opacity': 0.5
        })

    return horizontal_lines   

In [26]:
def gen_heat_map_from_arr2(gt_data, pred_data, obj_list=None, d_s=[]):
    hm_ = (gt_data*2) - pred_data

    color_tp = 'rgb(71, 137, 209)' 
    color_tn = 'rgb(104, 212, 202)'
    color_fp = 'rgb(202, 121, 59)' 
    color_fn = 'rgb(198, 59, 59)'
    color_grid = 'rgb(113,215,206)'
    
    fig = go.Figure()
    
    colorscale = [
        [0, color_fp],
        [0.33333, color_tn],
        [0.66667, color_tp],
        [1, color_fn]
    ]
    
    fig.update_layout(
        template="simple_white",
        width=920,
        height=500,
        margin=dict(l=0, r=0, t=10, b=5),
        title=dict(
            text="",
            font=dict(size=32, color="black"),
            x=0.5,
            y=0.995
    
        ),
        xaxis=dict(
            title_text="Frames", 
            tickmode='linear',
            showgrid=False, 
            dtick=1, 
            gridwidth=1
        ),
        yaxis=dict(
            title_text="Objects",  
            tickmode='linear',
            showgrid=False, 
            dtick=1, 
            gridwidth=1, 
        )
    )

    X = [f'{i}' for i in range(hm_.shape[1])]
    Y = [y.capitalize() for i, y in enumerate(obj_list)]
    
    fig.add_trace(
        go.Heatmap(
            z=hm_,
            x=X,
            y=Y,
            colorscale=colorscale,
            showscale=False,
            zmin=-1,
            zmax=2
        )
    )

    fig.add_trace(go.Scatter(
        x=[None],
        y=[None],
        mode='markers',
        name="TP",
        marker=dict(size=60, symbol='square', color=color_tp),
        showlegend=True
    ))
    fig.add_trace(go.Scatter(
        x=[None],
        y=[None],
        mode='markers',
        name="TN",
        marker=dict(size=60, symbol='square', color=color_tn),
        showlegend=True
    ))
    fig.add_trace(go.Scatter(
        x=[None],
        y=[None],
        mode='markers',
        name="FP",
        marker=dict(size=60, symbol='square', color=color_fp),
        showlegend=True
    ))
    fig.add_trace(go.Scatter(
        x=[None],
        y=[None],
        mode='markers',
        name="FN",
        marker=dict(size=60, symbol='square', color=color_fn),
        showlegend=True
    ))
    
    fig.update_layout(
        font_family="Courier New, monospace",
        font_color="black",
        title_font_family="Courier New, monospace",
        title_font_color="black",
        legend_title_font_color="black",
        legend=dict(orientation="h", y=1.14, x=0.1),
        font=dict(size=28),
    )

    fig.update_traces(
      colorbar_tickmode='array',
      colorbar_ticktext=['FP', 'TN', 'TP', 'FN'],
      colorbar_tickvals=[-1, 0, 1, 2],
      colorbar_tickangle=0,
      selector=dict(type='heatmap')
     )

    fig.update_xaxes(ticklabelposition='outside')
    
    fig.update_xaxes(
        tickson="boundaries",
        ticklen=5,
        tickangle=90
    )
    
    fig.update_layout(xaxis=dict(domain=[0,1]))
    layout_shapes_list = []
    layout_shapes_list.extend(get_vetical_axis_lines(X, color_grid))
    layout_shapes_list.extend(get_horizontal_axis_lines(Y, color_grid))   

    fig.update_layout(shapes = tuple(layout_shapes_list))
    
    hm_figs = fig

    return hm_figs

In [27]:
def gen_heat_map_model_only_from_arr2(pred_data, obj_list=None, d_s=[]):
    hm_ = pred_data # gt_data.mul(2).add(pred_data.mul(-1), fill_value=0)

    color_tp = 'rgb(71, 137, 209)' 
    color_tn = 'rgb(104, 212, 202)'
    color_fp = 'rgb(202, 121, 59)' 
    color_fn = 'rgb(198, 59, 59)'
    color_agreement = 'rgb(6, 200, 115)' # green
    color_disagreement = 'rgb(211, 6, 50)'
    color_grid = 'black'
    
    fig = go.Figure()
    
    colorscale = [
        [0, color_disagreement],
        [1, color_agreement]
    ]
    
    fig.update_layout(
        template="simple_white",
        xaxis=dict(title_text="Frames", tickmode='linear'),
        yaxis=dict(title_text="Objects",  tickmode='linear'),
        width=920,
        height=500,
        margin=dict(l=0, r=0, t=10, b=5),
        title=dict(
            text="",
            font=dict(size=32, color="black"),
            x=0.5,
            y=0.995
    
        )
    )
    X=[f'{i}' for i in range(hm_.shape[1])]
    Y=[y.capitalize() for i, y in enumerate(obj_list)]
    
    fig.add_trace(
        go.Heatmap(
            z=hm_,
            x=X,
            y=Y,
            colorscale=colorscale,
            showscale=False,
            zmin=0,
            zmax=1
        )
    )

    fig.add_trace(go.Scatter(
        x=[None],
        y=[None],
        mode='markers',
        name="Presence",
        marker=dict(size=60, symbol='square', color=color_agreement),
        showlegend=True
    ))
    fig.add_trace(go.Scatter(
        x=[None],
        y=[None],
        mode='markers',
        name="Absence",
        marker=dict(size=60, symbol='square', color=color_disagreement),
        showlegend=True
    ))
    
    fig.update_layout(
        font_family="Courier New, monospace",
        font_color="black",
        title_font_family="Courier New, monospace",
        title_font_color="black",
        legend_title_font_color="black",
        legend=dict(orientation="h", y=1.14, x=0.1),
        font=dict(size=28),
    )

    fig.update_traces(
      colorbar_tickmode='array',
      colorbar_ticktext=['Disagree', 'Agree'],
      colorbar_tickvals=[0, 1],
      colorbar_tickangle=0,
      selector=dict(type='heatmap')
     )

    fig.update_xaxes(ticklabelposition='outside')
    
    fig.update_xaxes(
        tickson="boundaries",
        ticklen=5,
        tickangle=90
    )
    # fig.update_layout(
    #      xaxis=dict(showgrid=True),
    #      yaxis=dict(showgrid=True)
    # )
    
    fig.update_layout(xaxis=dict(domain=[0, 1]))

    layout_shapes_list = []
    layout_shapes_list.extend(get_vetical_axis_lines(X, color_grid))
    layout_shapes_list.extend(get_horizontal_axis_lines(Y, color_grid))   

    fig.update_layout(shapes = tuple(layout_shapes_list))
    
    hm_figs = fig

    return hm_figs

In [42]:
def get_dum_pred_from_f1(gt_f__, org_f1, pred_ct, obj_list_a):
    dd_df = pd.read_csv(gt_f__)
    if limit_frame_count > 0:
        if len(list(dd_df.columns)) > limit_frame_count:
            dd_df = dd_df.iloc[: , :limit_frame_count]

    dd_df = dd_df.transpose()
    dd_df.columns = [x__.lower() for x__ in dd_df.iloc[0]]
    dd_df = dd_df.reindex(columns=obj_list_a).iloc[1:].transpose()
    dd_np = np.array(dd_df)
    
    unique, counts = np.unique(np.array(dd_df), return_counts=True)
    total_pred_c = sum(counts)
    pred_counts = dict(zip(unique, counts))
    fra_rep = [int(x) for x in str(Fraction(org_f1).limit_denominator()).split('/')]
    tp_frac = fra_rep[0]/2
    fp_plus_fn_frac = fra_rep[1] - fra_rep[0]
    dum_pred_dfs = []
    for p_tp in range(pred_counts[1], 0, -1):
        new_dd_np = np.zeros(dd_np.shape).astype(int)
        p_fp_plus_fn = round((fp_plus_fn_frac/tp_frac) * p_tp)
        p_fn = pred_counts[1] - p_tp
        p_fp = p_fp_plus_fn - p_fn
        p_tn = total_pred_c - (p_tp + p_fn + p_fp)
        p_f1 = (2 * p_tp)/(p_tp+p_tp+p_fp+p_fn)
        p_dev = abs((p_f1 - org_f1)/p_f1)
    
        if p_tn < 0 or p_dev > 0.08:
            continue
        
        pos_posit = np.argwhere(dd_np==1)
        pos_neg = np.argwhere(dd_np==0)
        p_rc_samp = random.sample(range(0, pos_posit.shape[0]), p_tp)
        
        try:
            n_rc_samp = random.sample(range(0, pos_neg.shape[0]), p_fp)
        except:
            print(pos_neg.shape, p_fp)
            if p_fp < 0:
                p_fp = 0
            n_rc_samp = random.sample(range(0, pos_neg.shape[0]), p_fp)
            
        tp_rrr, tp_ccc = pos_posit[:, 0][p_rc_samp], pos_posit[:, 1][p_rc_samp]
        fp_rrr, fp_ccc = pos_neg[:, 0][n_rc_samp], pos_neg[:, 1][n_rc_samp]
    
        new_dd_np[tp_rrr, tp_ccc] = 1
        new_dd_np[fp_rrr, fp_ccc] = 1

        new_dd_df = pd.DataFrame(new_dd_np, index=dd_df.index, columns=dd_df.columns).reset_index()
        new_dd_df = new_dd_df.rename(columns={"index": "Object"})

        dum_pred_dfs.append(new_dd_df)
        if len(dum_pred_dfs) >= pred_ct:
            break

    return dum_pred_dfs

In [29]:
def get_dum_pred_random(gt_f__, pred_ct, obj_list_a):
    dd_df = pd.read_csv(gt_f__)
    if limit_frame_count > 0:
        if len(list(dd_df.columns)) > limit_frame_count:
            dd_df = dd_df.iloc[: , :limit_frame_count]

    dd_df = dd_df.transpose()
    dd_df.columns = [x__.lower() for x__ in dd_df.iloc[0]]
    dd_df = dd_df.reindex(columns=obj_list_a).iloc[1:].transpose()
    dd_np = np.array(dd_df)
    
    dum_pred_dfs = []
    for d_pred_c in range(pred_ct):
        new_dd_np = np.zeros(dd_np.shape).astype(int)
        for rw in range(new_dd_np.shape[0]):
            for cl in range(new_dd_np.shape[1]):
                pred_val = random.randint(0, 1)
                new_dd_np[rw, cl] = pred_val

        new_dd_df = pd.DataFrame(new_dd_np, index=dd_df.index, columns=dd_df.columns).reset_index()
        new_dd_df = new_dd_df.rename(columns={"index": "Object"})

        dum_pred_dfs.append(new_dd_df)

    return dum_pred_dfs

In [44]:
import shutil

random.seed(100)
models = [
    'GPT4V', 'LLaVa', 'BLIP', 'GPV-1'
]

workbook = Workbook()
worksheet_s = workbook.active
worksheet_s.title = "Normal"
worksheet_cm = workbook.create_sheet("CM", 0)

col = 2

for gt_ind, gt in enumerate(tqdm(gts)):
    gt_f = os.path.join(gt_fol, f'{gt}.csv')

    v_ = int(gt.split('-')[1])
    s_ = int(gt.split('-')[3])

    for model in models:
        try:
            _, _, f1___, _ = calculate_model_ap_ar_af1(
                gt_fol, [f'{gt}.csv'], os.path.join(data_path, model), 
            )
        
            f1___ = float(f'{f1___:.4f}')
        except:
            print(model, gt)
            _, _, f1___, _ = calculate_model_ap_ar_af1(
                gt_fol, [f'{gt}.csv'], os.path.join(data_path, model), 
                obj_list=object_list
            )
        
            f1___ = float(f'{f1___:.4f}')

        print(f1___)

        try:
            d_df = get_dum_pred_from_f1(gt_f, f1___, 1, object_list)[0]
            export_df_path = f'./sim_f1_for_dashboard/{model}@Shadow'
            if not os.path.exists(export_df_path):
                os.makedirs(export_df_path)
                
            d_df.to_csv(os.path.join(export_df_path, f'{gt}.csv'), index=False)
        except Exception as e:
            print(e, gt, model)
            export_df_path = f'./sim_f1_for_dashboard/{model}@Shadow'
            if not os.path.exists(export_df_path):
                os.makedirs(export_df_path)
            try:
                shutil.copy(os.path.join(data_path, model, f'{gt}.csv'), export_df_path)
            except:
                print(f'{gt}.csv')

        

 14%|█▍        | 3/21 [00:00<00:00, 22.90it/s]

0.6274
0.418
0.386
0.2898
0.7065
0.4744
0.4467
0.3213
0.6933
0.4273
0.4216
0.318
0.6355
0.4802
0.4307
0.3569
0.6294
0.4757
0.4238
0.2828


 43%|████▎     | 9/21 [00:00<00:00, 21.62it/s]

0.7062
0.4599
0.4618
0.2429
(921, 2) 929
Sample larger than population or is negative video-4-segment-1 GPV-1
0.6963
0.4214
0.3844
0.3313
0.6323
0.3172
0.3005
0.201
0.6832
0.3864
0.4534
0.2133
0.6271
0.4593


 57%|█████▋    | 12/21 [00:00<00:00, 23.08it/s]/Users/ibk5106/anaconda3/envs/conda_pytorch/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ibk5106/anaconda3/envs/conda_pytorch/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ibk5106/anaconda3/envs/conda_pytorch/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1760: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted

0.3708
0.2986
0.6809
0.4618
0.4732
0.302
0.4381
0.2341
0.3742
0.2141
0.0
list index out of range video-9-segment-2 GPT4V
video-9-segment-2.csv
0.0
list index out of range video-9-segment-2 LLaVa
video-9-segment-2.csv
0.0
list index out of range video-9-segment-2 BLIP
0.0
list index out of range video-9-segment-2 GPV-1
0.0
list index out of range video-10-segment-1 GPT4V
video-10-segment-1.csv
0.0
list index out of range video-10-segment-1 LLaVa
video-10-segment-1.csv
0.0
list index out of range video-10-segment-1 BLIP
0.0
list index out of range video-10-segment-1 GPV-1
0.6725
0.6476
0.615
0.4533
0.7597
0.6636
0.6499
0.4859


100%|██████████| 21/21 [00:00<00:00, 25.11it/s]

0.7824
0.5491
0.4903
0.382
0.6822
0.4837
0.4266
0.3294
0.5317
0.5203
0.4213
0.3988
0.6239
0.6147
0.6
0.4461
0.6573
0.6455
0.5881
0.517


In [37]:
df = pd.read_csv('/Users/ibk5106/Desktop/research/vqa_accessibility/Dashboard-For-VQA/Dashboard Data/GPT4V/video-1-segment-5.csv')
df.isnull().values.any()

False